In [1]:
# This code is solution to a portfolio optimisation problem
# Description: Here, we select 5 most un-correlated stocks from the top 20 stocks contributing to nifty index.
# Selection method is such that, given a correlation matrix of 20 stocks, we select 5 such that their summation of correlation is minimum.

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import itertools

In [3]:
stock_tickers = ['HDFCBANK.NS', 'RELIANCE.NS', 'ICICIBANK.NS', 'INFY.NS', 'ITC.NS', 'TCS.NS', 'AXISBANK.NS', 'KOTAKBANK.NS', 'HINDUNILVR.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'BAJAJFINSV.NS', 'ASIANPAINT.NS', 'M&M.NS', 'MARUTI.NS', 'HCLTECH.NS', 'SUNPHARMA.NS', 'TITAN.NS', 'LTIM.NS', 'ADANIENT.NS']

In [4]:
stock_data = yf.download(stock_tickers, period='1y')

[*********************100%%**********************]  20 of 20 completed


In [5]:
stock_data

Adj Close                                           \
            ADANIENT.NS ASIANPAINT.NS  AXISBANK.NS BAJAJFINSV.NS   
Date                                                               
2022-11-14  4020.541260   3027.716797   850.132446   1716.545166   
2022-11-15  4055.773682   3053.599121   857.774719   1706.705566   
2022-11-16  3954.624268   3057.317871   854.528015   1683.879517   
2022-11-17  4016.143311   3045.963379   857.325134   1662.851685   
2022-11-18  4017.792725   3069.663818   858.324158   1652.662354   
...                 ...           ...          ...           ...   
2023-11-07  2233.350098   3033.350098  1020.900024   1569.800049   
2023-11-08  2259.949951   3095.250000  1021.700012   1575.500000   
2023-11-09  2214.649902   3080.199951  1021.900024   1582.250000   
2023-11-10  2205.100098   3076.449951  1029.199951   1596.000000   
2023-11-13  2213.649902   3085.600098  1025.349976   1585.199951   

                                                                               \
           BHARTIARTL.NS   HCLTECH.NS  HDFCBANK.NS HINDUNILVR.NS ICICIBANK.NS   
Date                                                                            
2022-11-14    818.141907  1051.631592  1596.738770   2420.139160   887.643555   
2022-11-15    831.430847  1056.320312  1600.643921   2419.843750   904.256775   
2022-11-16    837.353638  1057.564209  1614.386353   2439.882080   906.141235   
2022-11-17    842.281067  1045.459595  1599.803589   2422.305420   912.290649   
2022-11-18    833.471497  1055.602783  1595.601807   2445.642090   912.836121   
...                  ...          ...          ...           ...          ...   
2023-11-07    937.150024  1273.949951  1487.250000   2496.449951   948.099976   
2023-11-08    936.750000  1267.650024  1491.550049   2519.350098   935.750000   
2023-11-09    932.349976  1267.500000  1485.650024   2476.699951   937.349976   
2023-11-10    935.500000  1255.449951  1491.500000   2486.550049   938.599976   
2023-11-13    935.000000  1261.000000  1488.800049   2478.149902   934.450012   

                         ...    Volume                                 \
                INFY.NS  ...    ITC.NS KOTAKBANK.NS  LTIM.NS   M&M.NS   
Date                     ...                                            
2022-11-14  1523.786377  ...  17639889      2142267   664125  3794391   
2022-11-15  1532.148682  ...  15451411      1990292  1013747  3192070   
2022-11-16  1540.607300  ...   7604924      6437651   426335  2478281   
2022-11-17  1525.804810  ...   9994270      2408671   451474  2319979   
2022-11-18  1531.572021  ...   7838297      4272040   460492  4525364   
...                 ...  ...       ...          ...      ...      ...   
2023-11-07  1404.300049  ...   5124012      3599603   200866  1940908   
2023-11-08  1391.000000  ...   6979849      3642837   236117  1342202   
2023-11-09  1374.800049  ...   7785767      1943711   269946  4831782   
2023-11-10  1368.849976  ...   4079205      1634799   229733  4872228   
2023-11-13  1373.550049  ...   3744754      2855277   140187  1968032   

                                                                           
           MARUTI.NS RELIANCE.NS   SBIN.NS SUNPHARMA.NS   TCS.NS TITAN.NS  
Date                                                                       
2022-11-14    282822     4173395  12957268      1764788  1342074   565428  
2022-11-15    359888     3270388  10286417      1703050  1400708  1033296  
2022-11-16    350737     4484007  11555749      1485506  1748235   835064  
2022-11-17    409572     3074259   8167131      1758031  1417986  1265647  
2022-11-18    515615     2447425  12933675      1130682  1164540  1038751  
...              ...         ...       ...          ...      ...      ...  
2023-11-07    264935     3480102  17923281      1811831  1191144   433992  
2023-11-08    287006     3942247  15434808      1702301   992899   632305  
2023-11-09    310536     7256805  12434363      1849169  2112352  1

In [6]:
stock_closing_prices = stock_data['Adj Close']
returns = stock_closing_prices.pct_change().dropna()

In [7]:
correlation_matrix = returns.corr()

In [8]:
correlation_matrix

,ADANIENT.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJFINSV.NS,BHARTIARTL.NS,HCLTECH.NS,HDFCBANK.NS,HINDUNILVR.NS,ICICIBANK.NS,INFY.NS,ITC.NS,KOTAKBANK.NS,LTIM.NS,M&M.NS,MARUTI.NS,RELIANCE.NS,SBIN.NS,SUNPHARMA.NS,TCS.NS,TITAN.NS
ADANIENT.NS,1.000000,0.185245,0.125254,0.257626,0.100585,0.130884,0.160802,-0.018368,0.117109,0.111419,-0.076264,0.132808,0.090208,0.215893,0.175412,0.253568,0.381433,0.097667,0.142957,0.167799
ASIANPAINT.NS,0.185245,1.000000,0.050837,0.242292,0.090563,0.058586,0.117995,0.288172,0.182173,0.175713,0.077562,0.266360,0.060260,0.089877,0.129373,0.124210,0.142226,0.094088,0.210750,0.276484
AXISBANK.NS,0.125254,0.050837,1.000000,0.302487,0.251149,0.163800,0.274413,0.151347,0.483755,0.175286,0.163235,0.249314,0.155736,0.274665,0.258778,0.276746,0.451437,-0.035120,0.142379,0.183251
BAJAJFINSV.NS,0.257626,0.242292,0.302487,1.000000,0.125357,0.136276,0.262084,-0.030443,0.273947,0.180857,0.086818,0.257436,0.189588,0.290609,0.265077,0.292282,0.447778,0.115266,0.221440,0.442765
BHARTIARTL.NS,0.100585,0.090563,0.251149,0.125357,1.000000,0.181056,0.277194,0.131933,0.303538,0.207160,0.222343,0.354370,0.160538,0.187597,0.193570,0.222967,0.211788,0.153550,0.160466,0.118894
HCLTECH.NS,0.130884,0.058586,0.163800,0.136276,0.181056,1.000000,0.212429,0.172357,0.165795,0.595291,0.090121,0.176050,0.544367,0.165613,0.207897,0.213256,0.162210,0.122037,0.615957,0.047330
HDFCBANK.NS,0.160802,0.117995,0.274413,0.262084,0.277194,0.212429,1.000000,0.094379,0.387829,0.295106,0.088704,0.311857,0.239850,0.236972,0.067584,0.324569,0.345236,0.128951,0.274999,0.203840
HINDUNILVR.NS,-0.018368,0.288172,0.151347,-0.030443,0.131933,0.172357,0.094379,1.000000,0.122826,0.222190,0.360067,0.151336,0.081831,0.201786,0.176619,0.149670,0.073407,0.188707,0.156949,0.158276
ICICIBANK.NS,0.117109,0.182173,0.483755,0.273947,0.303538,0.165795,0.387829,0.122826,1.000000,0.240914,0.236828,0.468807,0.210022,0.215229,0.223120,0.335996,0.455942,-0.041069,0.201257,0.179787
INFY.NS,0.111419,0.175713,0.175286,0.180857,0.207160,0.595291,0.295106,0.222190,0.240914,1.000000,0.142293,0.077517,0.557725,0.191087,0.113816,0.319788,0.114395,0.198341,0.675130,0.072280


In [9]:
# Generating iterables for looping.

combinations = itertools.combinations(range(20), 5)

In [10]:
min_summation = 20
selected_stocks = []

In [11]:
# iterating through iterables to get 5 stocks such that their summation of correlation is minimum.

for combo in combinations:
    for i, j in itertools.combinations(combo,2):
    print(i)
    summation = sum(correlation_matrix.iloc[i, j] for i, j in itertools.combinations(combo, 2))
    if summation < min_summation:
        min_summation = summation
        selected_stocks = combo

for i, j in itertools.combinations(combo,2):
    print(i)

AttributeError: module 'itertools' has no attribute 'combinaitons'

In [ ]:
# making the list of stocks

selected_stock_tickers = []
for i in selected_stocks:
    selected_stock_tickers.append(correlation_matrix.columns[i])


In [ ]:
selected_stock_tickers

In [ ]:
min_summation